In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

import hydrofunctions as hf

import fbprophet 

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [50]:
site = '09081000'
start = '2010-01-23'
end = '2020-01-23'

herring = hf.NWIS(site, 'dv', start, end)

In [51]:
herring.get_data()

In [52]:
my_dict = herring.json()
df = hf.extract_nwis_df(my_dict)
df.columns

Index(['USGS:09081000:00060:00003', 'USGS:09081000:00060:00003_qualifiers'], dtype='object')

In [53]:
df.head()

,USGS:09081000:00060:00003,USGS:09081000:00060:00003_qualifiers
datetime,,
2010-01-23,233.0,A
2010-01-24,229.0,A
2010-01-25,205.0,A e
2010-01-26,191.0,A e
2010-01-27,248.0,A e


In [54]:
df.plot()
plt.savefig('RRFork1.png')

In [56]:
df.rename(index=str, columns = {"datetime" : "ds", 
                                "USGS:09081000:00060:00003" : "y", 
                                "USGS:09081000:00060:00003_qualifiers" : "flags"}, 
         inplace = True)

df.head()

,y,flags
datetime,,
2010-01-23 00:00:00,233.0,A
2010-01-24 00:00:00,229.0,A
2010-01-25 00:00:00,205.0,A e
2010-01-26 00:00:00,191.0,A e
2010-01-27 00:00:00,248.0,A e


In [57]:
df_prophet = fbprophet.Prophet()
df_prophet.fit(df)

ValueError: Dataframe must have columns 'ds' and 'y' with the dates and values respectively.